## gRPC Method

In [13]:
import grpc
import numpy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import pandas as pd
import numpy as np
from PIL import Image

In [14]:
channel = grpc.insecure_channel('localhost:2525')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = 'detect_model'
request.model_spec.signature_name = 'serving_default'

In [15]:

img = Image.open('cam_image_test.JPG')
im_width, im_height = img.size
img= np.array(img)
img = img.reshape(1, im_height, im_width, 3)

# center pixel data
#img = img.astype('float32')

In [16]:
img.shape, im_height, im_width, 

((1, 540, 960, 3), 540, 960)

In [17]:
request.inputs['input_tensor'].CopyFrom(tf.make_tensor_proto(img, shape=[img.shape[0],im_height,im_width,3]))
result = stub.Predict(request, 10.0)

In [18]:
boxes = np.array(result.outputs['detection_boxes'].float_val)
boxes= boxes.reshape(10,4)
boxes.shape
df1= pd.DataFrame({'ymin': boxes[:,0], 'xmin': boxes[:,1], 'ymax': boxes[:,2], 'xmax': boxes[:,3]})
df1.head()

,ymin,xmin,ymax,xmax
0,0.041901,0.532943,0.472991,0.731399
1,0.485389,0.531159,0.993194,0.750819
2,0.041901,0.532943,0.472991,0.731399
3,0.442336,0.303715,0.975175,0.514077
4,0.052034,0.313935,0.505633,0.511166


In [19]:
scores = np.array(result.outputs['detection_scores'].float_val)
scores.shape
df2=pd.DataFrame({'detection_scores': scores})
df2.head()

,detection_scores
0,0.562568
1,0.544382
2,0.518533
3,0.506171
4,0.501039


In [20]:
classes = np.array(result.outputs['detection_classes'].float_val)
classes.shape
df3=pd.DataFrame({'detection_classes': classes})
df3.head()

,detection_classes
0,3.0
1,3.0
2,5.0
3,5.0
4,3.0


In [21]:
df_final= pd.concat([df1, df2, df3], axis=1)
df_final.head()

,ymin,xmin,ymax,xmax,detection_scores,detection_classes
0,0.041901,0.532943,0.472991,0.731399,0.562568,3.0
1,0.485389,0.531159,0.993194,0.750819,0.544382,3.0
2,0.041901,0.532943,0.472991,0.731399,0.518533,5.0
3,0.442336,0.303715,0.975175,0.514077,0.506171,5.0
4,0.052034,0.313935,0.505633,0.511166,0.501039,3.0


In [22]:
df_final=df_final[df_final['detection_scores']>0.30]
df_array=df_final.to_numpy()

In [23]:
#Tensorflow also has non max suppression algorithm
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    y1 = boxes[:,0]
    x1 = boxes[:,1]
    y2 = boxes[:,2]
    x2 = boxes[:,3]
    d_score = boxes[:,4]

    
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        scores_ = d_score[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
        np.where(overlap > overlapThresh)[0])))
        # return only the bounding boxes that were picked using the
        # integer data type
    return boxes[[pick]]

In [24]:
print(df_array.shape)
pick= non_max_suppression_fast(df_array, 0.9)
print(pick.shape)

(8, 6)
(4, 6)


In [25]:
pick= pd.DataFrame(pick, columns=['ymin', 'xmin', 'ymax', 'xmax', 'detection_scores',	'detection_classes'])

In [34]:
pick

,ymin,xmin,ymax,xmax,detection_scores,detection_classes
0,0.485389,0.531159,0.993194,0.750819,0.494667,king
1,0.466983,0.296644,0.986787,0.520197,0.463392,jack
2,0.052034,0.313935,0.505633,0.511166,0.501039,jack
3,0.041901,0.532943,0.472991,0.731399,0.518533,king


In [32]:
pd.set_option('mode.chained_assignment', 'raise')
class_dict={1:'nine', 2:'ten', 3:'jack', 4:'queen', 5:'king', 6:'ace'}
for i in range(len(pick)):
    pick.loc[i,'detection_classes']=class_dict[pick['detection_classes'][i]]

In [6]:
def plot_rec(coor, img, label='', color=(255,0,0),text_place='up'):
    x1, y1, x2, y2 = coor
    draw_img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness=3)
    if text_place== 'up':
        cv2.putText(draw_img, label, (int(x1), int(y1)), cv2.FONT_HERSHEY_DUPLEX, 1, color, 2)
    if text_place== 'down':
        cv2.putText(draw_img, label, (int(x1), int(y2)), cv2.FONT_HERSHEY_DUPLEX, 1, color, 2)

    return draw_img

In [20]:
import cv2
cv_img = cv2.imread('cam_image_test.jpg')
class_dict={1:'nine', 2:'ten', 3:'jack', 4:'queen', 5:'king', 6:'ace'}
for i ,j in pick.iterrows():
    x1 = j.xmin*im_width
    y1= j.ymin*im_height
    x2= j.xmax*im_width
    y2= j.ymax*im_height
    label= class_dict[j.detection_classes]
    anno_img = plot_rec((x1, y1, x2, y2), cv_img, label, color= (0,0,255),text_place= 'up')
cv2.imwrite('camlana_out.jpg', anno_img)

True

## REST API Method

In [22]:
import requests
import pandas as pd
import base64

In [36]:
url = 'https://localhost:1337/object'
my_img = {'image': open('cam_image.JPG', 'rb')}
r = requests.post(url, files=my_img, verify=False)

In [37]:
result= pd.DataFrame.from_dict(r.json()['results'])

In [38]:
result

,detection_classes,detection_scores,xmax,xmin,ymax,ymin
0,king,0.494667,720.786381,509.912395,536.324944,262.110234
1,jack,0.463392,499.388752,284.778385,532.865217,252.170863
2,jack,0.501039,490.719395,301.377611,273.041689,28.098427
3,king,0.518533,702.142868,511.625404,255.415167,22.626496


In [11]:
import cv2
cv_img = cv2.imread('cards.JPG')
for i ,j in result.iterrows():
    x1 = j.xmin
    y1= j.ymin
    x2= j.xmax
    y2= j.ymax
    label= j.detection_classes
    anno_img = plot_rec((x1, y1, x2, y2), cv_img, label, color= (0,0,255),text_place= 'down')
cv2.imwrite('cards_out.jpg', anno_img)

True